In [1]:
import nltk
import tensorflow as tf
import keras
import string
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
nltk.download('brown')

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [41]:
tag_sent = nltk.corpus.brown.tagged_sents()

# po=set([y[1] for x in tag_sent for y in x])
sentences=[[x[0] for x in y] for y in tag_sent]

tags=[[x[1] for x in y] for y in tag_sent]
print(sentences[0])

# tmp1,tmp2=[],[]
# for x,x1 in zip(sentences,tags):
#     new_sent=[]
#     new_punc=[]
#     for y,y1 in zip(x,x1):
#     # if y in '!"#$%&()*+,-/:;<=>?@[\\]^_\'`{|}~\t\n':
#         if len(y.translate({ord(x):None for x in '!."#$%&()*+,-/:;<=>?@[\\]^_\'`{|}~\t\n'}))==0:
#             pass
#         else:
#             new_sent.append(y)
#             new_punc.append(y1)
#             tmp1.append(new_sent)
#             tmp2.append(new_punc)
# sentences=tmp1
# tags=tmp2

def concat(li):
    sent=''
    for x in li:
        if len(sent)!=0:
            sent=sent+' '
        sent=sent+x
    return sent
MAX_LEN=60
sentences=[concat(x).lower() for x in sentences]
print(sentences[0])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
the fulton county grand jury said friday an investigation of atlanta's recent primary election produced `` no evidence '' that any irregularities took place .


In [42]:
NUM_WORDS=50000
tok=keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,oov_token='<OOV>',filters='\t\n')
tok.fit_on_texts(sentences)
# print(tok.get_config())
# print(tok.word_index)
sentences=tok.texts_to_sequences(sentences)

sentences=keras.preprocessing.sequence.pad_sequences(sentences,padding='post',maxlen=MAX_LEN)
# print(a[4])

# print(len(set([x for y in tags for x in y]+['$'])))

print('koko')
tags=keras.preprocessing.sequence.pad_sequences(tags,padding='post',value='$',maxlen=MAX_LEN,dtype=object).tolist()
print(tags[0])
# NUM_TAGS=set([x for y in tags for x in y]+['$'])
print(len(tags[0]))
tok_tag=keras.preprocessing.text.Tokenizer(oov_token='<UNK>',filters='\t\n')


tok_tag.fit_on_texts(tags)
num_tags=len(tok_tag.word_index)
tags=tok_tag.texts_to_sequences(tags)
tags=np.array(tags)
print(tags[0])
print(len(tags[0]))

koko
['AT', 'NP-TL', 'NN-TL', 'JJ-TL', 'NN-TL', 'VBD', 'NR', 'AT', 'NN', 'IN', 'NP$', 'JJ', 'NN', 'NN', 'VBD', '``', 'AT', 'NN', "''", 'CS', 'DTI', 'NNS', 'VBD', 'NN', '.', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$']
60
[ 5 42 23 41 23 15 60  5  3  4 50  6  3  3 15 30  5  3 31 16 48  9 15  3
  7  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2]
60


In [47]:
print(tok_tag.word_index['\'\''])

31


In [48]:
print(len(tags))

57340


In [49]:
with strategy.scope():
    model = Sequential()
    model.add(InputLayer(input_shape=(MAX_LEN,), dtype='int32'))
    # model.add(InputLayer(input_shape=(MAX_LEN,)))
    model.add(Embedding(NUM_WORDS+1, 128,trainable=True))
    model.add(Bidirectional(LSTM(128, return_sequences=True,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-3))))
    model.add(TimeDistributed(Dense(num_tags+1,activation='softmax',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-3))))

model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(0.001),metrics=['accuracy'])

model.summary()
print(type(sentences))
print(type(sentences[0]))
print(type(tags[0]))
print(type(tags))


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 60, 128)           6400128   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 60, 256)           263168    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 60, 473)           121561    
Total params: 6,784,857
Trainable params: 6,784,857
Non-trainable params: 0
_________________________________________________________________
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [50]:
model.fit(x=sentences,y=tags,validation_split=0.1,epochs=5,batch_size = 10 * strategy.num_replicas_in_sync)

Epoch 1/5
646/646 [==============================] - 15s 24ms/step - loss: 1.4903 - accuracy: 0.7226 - val_loss: 0.8664 - val_accuracy: 0.8069
Epoch 2/5
646/646 [==============================] - 11s 17ms/step - loss: 0.8168 - accuracy: 0.8522 - val_loss: 0.6185 - val_accuracy: 0.9161
Epoch 3/5
646/646 [==============================] - 11s 17ms/step - loss: 0.6313 - accuracy: 0.9188 - val_loss: 0.5271 - val_accuracy: 0.9438
Epoch 4/5
646/646 [==============================] - 11s 17ms/step - loss: 0.5505 - accuracy: 0.9388 - val_loss: 0.4817 - val_accuracy: 0.9522
Epoch 5/5
646/646 [==============================] - 11s 17ms/step - loss: 0.5009 - accuracy: 0.9490 - val_loss: 0.4473 - val_accuracy: 0.9590
